In [18]:
from keras.applications import ResNet50, InceptionV3
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, Flatten, Dense, Conv1D, Reshape, Conv2D, MaxPool1D, \
                         Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from skimage.transform import resize
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
def min_max_standard_feature(X):
    X_standard = []
    for feature in X:
        feature_max = np.max(feature)
        feature_min = np.min(feature)
        feature_standard = (feature - feature_min) / (feature_max - feature_min)
        X_standard.append(feature_standard)
    return X_standard

In [3]:
def trans_feature_to_3channel(X, target_img_shape):
    X_3channel = []
    for feature in X:
        feature_3channel = np.zeros(shape=(feature.shape[0], feature.shape[1], 3))
        feature_3channel[:,:,0] = feature
        feature_3channel[:,:,1] = feature
        feature_3channel[:,:,2] = feature
        feature_3channel = resize(feature_3channel, output_shape=target_img_shape)
        X_3channel.append(feature_3channel)
    return np.array(X_3channel, dtype=np.float32)

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [41]:
# add a global spatial average pooling layer
x = base_model.get_layer('activation_3').output
x = Conv2D(1, 1, activation='relu')(x)
x = Reshape(target_shape=(55, 55))(x)

x= Conv1D(64, 3, activation='relu')(x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPool1D(pool_size=2)(x)

x= Conv1D(64, 3, activation='relu')(x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPool1D(pool_size=2)(x)

x = Flatten()(x)
x = Dropout(0.5)(x)
# # let's add a fully-connected layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(10, activation='softmax')(x)
#freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [37]:
X_melspec = np.load('without_split_feature/stft_feature.npy')
y = np.load('without_split_feature/onehot_labels.npy')
X_melspec_standard = min_max_standard_feature(X_melspec)
X_melspec_3channel = trans_feature_to_3channel(X_melspec_standard, (224, 224, 3))
X_train, X_val, y_train, y_val = train_test_split(X_melspec_3channel, y, test_size=0.3)

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [42]:
model.summary()
opt = Adam(lr=0.001)
model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val), verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 230, 230, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 64)      9472      
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 112, 112, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
res2a_branch2a (Conv2D)      (None, 55, 55, 64)        4160      
__________